# ECE W382V MACHINE PROGRAMMING

# Homework 2 - due Wednesday July 27 2022 at 7:00am

For this homework you will hand in (upload) to canvas your code named ``hw2_YourEID.ipynb``.

__Before submitting__, please reset your kernel and rerun everything from the beginning (Kernel >> Restart and Run All) and ensure your code does __NOT__ give ANY error.

For programming tasks, make sure that your code can run using Python 3.5+. If you cannot complete a problem, include as much pseudocode as possible in the form of __python comment__ for partial credit. Please do NOT leave imcomplete code in your homework, please wrap them up in the comment.

For non-programming tasks, make sure the cell is a markdown cell. Information can be found about Markdown cells for Jupyter Notebooks here: https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html

Collaboration: you are free to discuss the homework assignments with other students. However, all the code you write must be your own!



### Please list any person you discussed this assignment with:

- 


# Problem 1: Naive Bayes (15 points)

We'll do a bit of manual parameter estimation here.

## **(a)** (8 points) 
Calculate the sufficient parameters for Naive Bayes using the data in the figure below, that
is, the prior class probabilities and the conditional probabilities for all of the symbols.

doc | X | Y
-- | --| --
d1 | a b b b c b | +
d2 | c a c c c b | -
d3 | c c c b | -
d4 | b a b b b | +
d5 | b c a b b | ???


Prior Class Probabilities:
- $P(+) = \frac{1}{2}$
- $P(-) = \frac{1}{2}$

Conditional Symbol Probabilities:
- $P(a|+) = \frac{2}{11}$
- $P(a|-) = \frac{1}{10}$
---
- $P(b|+) = \frac{8}{11}$
- $P(b|-) = \frac{2}{10}$
---
- $P(c|+) = \frac{1}{11}$
- $P(c|-) = \frac{7}{10}$


## **(b)** (7 points) 
Using these, calculate the most likely class (pos or neg) for the unlabeled example (d5, labeled ???).

- $P(+|b, c, a, b, b)$ = $P(+) P(b|+)^3 P(c|+) P(a|+)$ = $\frac{1}{2} * (\frac{8}{11})^3 * \frac{1}{11} * \frac{2}{11}$ = 0.00318
---
- $P(-|b, c, a, b, b)$ = $P(-) P(b|-)^3 P(c|-) P(a|-)$ = $\frac{1}{2} * (\frac{2}{10})^3 * \frac{7}{10} * \frac{1}{10}$ = 0.00028
---
\+  is the most likely label for d5

In [1]:
from sklearn.naive_bayes import MultinomialNB
import numpy as np

X = [
    [1, 4, 1],
    [1, 1, 4],
    [0, 1, 3],
    [1, 4, 0],
]
X = np.transpose(X)
y = ['+', '-','-','+']

clf = MultinomialNB(alpha=0)
clf.fit(np.transpose(X), y)

print("Classes (logs of the fractions above)\n", clf.class_log_prior_, "\n")
print("Features (logs of the fractions above)\n", clf.feature_log_prob_, "\n")

print("Prediction:")
clf.predict([[1, 3, 1]])

Classes (logs of the fractions above)
 [-0.69314718 -0.69314718] 

Features (logs of the fractions above)
 [[-1.70474809 -0.31845373 -2.39789527]
 [-2.30258509 -1.60943791 -0.35667494]] 

Prediction:


/home/jboy/.local/lib/python3.8/site-packages/sklearn/naive_bayes.py:591: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


array(['+'], dtype='<U1')

# Problem 2: Language Modeling (85 points)

In this problem, we will implement our own Java language models!

### Data

We will be working with a few different files under the `data` subdirectory. 
- `apache_spark.large.txt`
- `apache_spark.short.txt`
- `google_gson.large.txt`
- `test.java.txt`

You can use the "short" text files to do testing and debugging.

## (a) Find n-grams. (10 points) 

Create a function `make_ngram_tuples(words, n)` that returns a sequence of all n-grams seen in the input, in order.  The function returns a sequence of tuples where each tuple is of the form `(context, word)`.  The context is a tuple of the preceding n−1 words for each word; if the number of preceding words is smaller than n−1, place a `<s>` symbol in place of each missing context.  Close each sentence with an additional token `</s>`.  You can assumen n>1, that is, we are interested in enumerating bigrams, trigrams etc, and not unigrams.

For now, you can use any random string to test this function.

```
>>> samples = [’she’, ’eats’, ’happily’ ’.’]
>>> make_ngram_tuples(samples, 2)
[((’<s>’,), ’she’), ((’she’,), ’eats’), ((’eats’,), ’happily’), ((’happily’,), ’.’), ((’.’,), ’</s>’)]
>>> make_ngram_tuples(samples, 3)
[((’<s>’, ’<s>’), ’she’), ((’<s>’, ’she’), ’eats’), ((’she’, ’eats’), ’happily’),((’eats’, ’happily’), ’.’), ((’happily’, ’.’), ’</s>’)]
```

In [2]:
from typing import List, Tuple  # Do not change

In [25]:
def make_ngram_tuples(words, n) -> List[Tuple]:
    tuples = []
    for t in zip(*[['<s>'] * i + words + ['</s>'] for i in reversed(range(n))]):
        tuples.append((t[:n-1], t[n-1]))
        
    return tuples

In [28]:
# Do not change
samples = ["she", "eats", "happliy", "."]
make_ngram_tuples(samples, 5)

[(('<s>', '<s>', '<s>', '<s>'), 'she'),
 (('<s>', '<s>', '<s>', 'she'), 'eats'),
 (('<s>', '<s>', 'she', 'eats'), 'happliy'),
 (('<s>', 'she', 'eats', 'happliy'), '.'),
 (('she', 'eats', 'happliy', '.'), '</s>')]

##### (b)  Building an n-gram language model

We are now ready to estimate a bigram language model from the training data.  We will use **Laplace smoothing (add-1)**.

### (b1) Process the training file (8 points)

We will first need to transform a file of tokenized functions into a list of "function", where each "function" is a list of *lower-cased* tokens. Since we have already provided the tokenized functions (see files in the `data/` directory), you only need to split the given string by space. Implement a function `process_code(codefile)` to do so. Assume that the path of `codefile` is of a form like `data/apache_spark.short.txt`, i.e., relative paths. 

```
>>> tokenized_funcs = process_code("data/apache_spark.short.txt")
>>> tokenized_funcs[1]
['public', 'file', 'get', 'file', '(', ')', '{', 'return', 'file', ';', '}']
```

In [5]:
def process_code(codefile) -> List[List[str]]:
    return [l.split() for l in open(codefile)]

In [6]:
# Do not change
tokenized_funcs = process_code("../datasets/hw2/apache_spark.short.txt")
tokenized_funcs[1]

['public', 'file', 'get', 'file', '(', ')', '{', 'return', 'file', ';', '}']

### (b2) Vocabulary (8 points).

The language model ought to be able to handle words not seen in the training data.  Such words will most certainly appear if the LM is used in some application to estimate the likelihood of new data.   There  are  many  ways  to  incorporate  unknown  vocabulary  in  a  language  model.   In  this problem, we will take all words that appear only once and replace them with the symbol `<UNK>`. Thus, at test time, if we encounter an out-of-vocabulary word, we can resort to replacing the word with `<UNK>` which has well-formed probabilities.

First, implement a function `get_vocab(tokenized_funcs)` where the parameter `tokenized_funcs` is a list of tokenized "function" (where each "function" is a list of lower-cased tokens) returned by the function `process_code`. The function `get_vocab` will return a `set` of all unique words in `tokenized_funcs` that appeared **more than** once.

```
>>> vocab = get_vocab(tokenized_funcs)
>>> len(vocab)
1375
```

In [7]:
from collections import Counter

def get_vocab(tokenized_funcs) -> set:
    counter = Counter(token for function in tokenized_funcs for token in function)
    return {token for token, count in counter.items() if count > 1}

In [8]:
# Do not change
vocab = get_vocab(tokenized_funcs)
len(vocab)

1375

### (b3) Process unknown words (8 points).

Write a function `process_unk(tokenized_funcs, vocab)` that takes in (1) tokenized functions returned by `process_code`, and (2) a vocabulary (set of words) returned by `get_vocab`. The function returns a list of tokenized functions that is the same as `tokenized_funcs` except that all words appearing only once will be replaced by the symbol `<UNK>`.

```
>>> processed_funcs[3]
['@', 'override', 'public', 'operation', 'handle', 'get', 'table', 'types', '(', ')', 'throws', 'hive', 'sqlexception', '{', 'acquire', '(', 'true', ')', ';', 'operation', 'manager', 'operation', 'manager', '=', 'get', 'operation', 'manager', '(', ')', ';', 'get', 'table', 'types', 'operation', 'operation', '=', 'operation', 'manager', '.', 'new', 'get', 'table', 'types', 'operation', '(', 'get', 'session', '(', ')', ')', ';', 'operation', 'handle', 'op', 'handle', '=', 'operation', '.', 'get', 'handle', '(', ')', ';', 'try', '{', 'operation', '.', 'run', '(', ')', ';', 'op', 'handle', 'set', '.', 'add', '(', 'op', 'handle', ')', ';', 'return', 'op', 'handle', ';', '}', 'catch', '(', 'hive', 'sqlexception', 'e', ')', '{', 'operation', 'manager', '.', 'close', 'operation', '(', 'op', 'handle', ')', ';', 'throw', 'e', ';', '}', 'finally', '{', 'release', '(', 'true', ')', ';', '}', '}']
```

In [9]:
def process_unk(tokenized_funcs, vocab) -> List[List[str]]:
    replace_unk = lambda func: ['<UNK>' if token not in vocab else token for token in func]
    return [replace_unk(func) for func in tokenized_funcs]

In [10]:
# Do not change
processed_unk = process_unk(tokenized_funcs, vocab)
processed_unk[3]

['@',
 'override',
 'public',
 'operation',
 'handle',
 'get',
 'table',
 'types',
 '(',
 ')',
 'throws',
 'hive',
 'sqlexception',
 '{',
 'acquire',
 '(',
 'true',
 ')',
 ';',
 'operation',
 'manager',
 'operation',
 'manager',
 '=',
 'get',
 'operation',
 'manager',
 '(',
 ')',
 ';',
 'get',
 'table',
 'types',
 'operation',
 'operation',
 '=',
 'operation',
 'manager',
 '.',
 'new',
 'get',
 'table',
 'types',
 'operation',
 '(',
 'get',
 'session',
 '(',
 ')',
 ')',
 ';',
 'operation',
 'handle',
 'op',
 'handle',
 '=',
 'operation',
 '.',
 'get',
 'handle',
 '(',
 ')',
 ';',
 'try',
 '{',
 'operation',
 '.',
 'run',
 '(',
 ')',
 ';',
 'op',
 'handle',
 'set',
 '.',
 'add',
 '(',
 'op',
 'handle',
 ')',
 ';',
 'return',
 'op',
 'handle',
 ';',
 '}',
 'catch',
 '(',
 'hive',
 'sqlexception',
 'e',
 ')',
 '{',
 'operation',
 'manager',
 '.',
 'close',
 'operation',
 '(',
 'op',
 'handle',
 ')',
 ';',
 'throw',
 'e',
 ';',
 '}',
 'finally',
 '{',
 'release',
 '(',
 'true',
 ')',
 ';',

In [11]:
x = process_unk([["throw", "HOLA", "true"]], vocab)
x[0]

['throw', '<UNK>', 'true']

### (b4) N-gram frequencies (10 points).

We now need to get the frequency counts -- which will allow us to calculate the conditional frequency distribution for our language model. Write a function `get_freq_dist(tokenized_funcs, n)` that takes in (1) a list of tokenized functions (such as one returned by `process_unk`), and (2) the number `n` that denotes the order of the n-gram (`n=2` means bigram, `n=3` means trigram, etc). The function will return a dictionary `freqdict` (`freqdict` can be a `Counter`) such that `freqdict[context][token]` records the number of times `token` follows `context`. You can think of `context` as the first element of a tuple in a list returned by `make_ngram_tuples`, and `token` as the second element of that tuple.

```
>>> freqdict = get_freq_dict(processed_unk, 2)
>>> freqdict[('public',)]
Counter({'void': 285, 'static': 70, 'boolean': 57, 'int': 39, 'string': 33, 'abstract': 16, 'long': 15, 'final': 13, 'get': 13, 'tget': 12, 'object': 8, 'org': 7, '_': 7, 'java': 7, 'short': 6, 'unsafe': 6, 'synchronized': 6, 'utf8': 6, 'table': 5, 'operation': 4, 'double': 4, 'float': 4, 'byte': 4, 'tclose': 4, 'texecute': 4, '<': 3, 'managed': 3, 'stream': 3, 'spark': 3, 'topen': 3, 'cancel': 3, 't': 3, 'close': 3, 'columnar': 3, 'optional': 2, 'row': 2, 'tstring': 2, 'map': 2, 'ttype': 2, 'tfetch': 2, 'fetch': 2, 'tcolumn': 2, 'ti16': 2, 'tstatus': 2, 'channel': 2, 'list': 2, 'circular': 2, 'tcancel': 2, 'calendar': 2, 'scan': 2, 'iterator': 2, 'file': 1, 'tmap': 1, '<UNK>': 1, 'transport': 1, 'avro': 1, 'server': 1, 'bloom': 1, 'tunion': 1, 'struct': 1, 'id': 1, 'mutable': 1, 'set': 1, 'tbool': 1, 'metadata': 1, 'parquet': 1, 'toperation': 1, 'tbyte': 1, 'integer': 1, 'tsession': 1, 'renew': 1, 'shuffle': 1, 'tstruct': 1, 'rpc': 1, 'trenew': 1, 'param': 1, 'decimal': 1, 'tprotocol': 1, 'metric': 1, 'nesting3': 1, 'location': 1, 'message': 1, '(': 1, 'execute': 1})
```

In [12]:
from collections import defaultdict

def get_freq_dict(tokenized_funcs, n) -> dict:
    vocab = get_vocab(tokenized_funcs)
    tokenized_funcs_with_unk = process_unk(tokenized_funcs, vocab)
    
    freqdict = defaultdict(Counter)
    
    for func in tokenized_funcs_with_unk:
        for context, token in make_ngram_tuples(func, n):
            freqdict[context][token] += 1
    
    return freqdict

In [13]:
# Do not change this
freqdict = get_freq_dict(processed_unk, 2)
freqdict[('public',)]

Counter({'void': 285,
         'file': 1,
         'string': 33,
         'operation': 4,
         'org': 7,
         'boolean': 57,
         'static': 70,
         '_': 7,
         'double': 4,
         'float': 4,
         '<': 3,
         'managed': 3,
         'stream': 3,
         'optional': 2,
         'spark': 3,
         'tmap': 1,
         'int': 39,
         'topen': 3,
         '<UNK>': 1,
         'final': 13,
         'short': 6,
         'unsafe': 6,
         'synchronized': 6,
         'row': 2,
         'byte': 4,
         'transport': 1,
         'long': 15,
         'avro': 1,
         'abstract': 16,
         'tget': 12,
         'object': 8,
         'cancel': 3,
         't': 3,
         'java': 7,
         'tstring': 2,
         'utf8': 6,
         'tclose': 4,
         'server': 1,
         'map': 2,
         'table': 5,
         'bloom': 1,
         'ttype': 2,
         'get': 13,
         'tfetch': 2,
         'tunion': 1,
         'struct': 1,
         'id': 

In [14]:
freqdict[('}',)]

Counter({'platform': 1,
         '</s>': 788,
         'else': 126,
         'first': 16,
         'sb': 8,
         'catch': 62,
         'finally': 13,
         '}': 235,
         's': 1,
         'case': 1,
         'return': 64,
         'byte': 2,
         'try': 7,
         'current': 3,
         'this': 5,
         'boolean': 13,
         'buf': 1,
         'final': 4,
         ';': 7,
         '@': 4,
         'int': 8,
         'if': 52,
         'from': 2,
         'secret': 1,
         'block': 1,
         '"': 12,
         'tstatus': 1,
         'quoted': 1,
         'break': 43,
         'struct': 4,
         'switch': 23,
         'iprot': 31,
         'last': 4,
         'row': 4,
         'super': 3,
         ')': 29,
         'check': 1,
         'for': 5,
         'shuffle': 1,
         'map': 2,
         'verify': 2,
         'sorter': 4,
         ',': 16,
         'connection': 1,
         'throw': 15,
         'spark': 2,
         'oprot': 15,
         'logger': 1,

### (b5) The langauge model

We are now ready to put everything together and make our langauge model! Below we have written the function `build_ngram_model(codefile, n)` that takes in a text file, the value `n` that signals the order of our n-gram, and returns a language model in the form of a `namedtuple`. All we need to do is to call the various functions you've implemented so far in (c)! (**No implementation required here**).

A `namedtuple` is a versatile data structure that allows one to associate multiple data fields with one variable. Below, we created a `namedtuple` called `LanguageModel`; this `LanguageModel` consists of the following information: the n-gram order `n`, the frequency distribution dictionary `fd`, and the vocabulary (as a `set` of words) `vocab`. Now, after we make a `LanguageModel`, we will be able to access these fields using the `dot` operator, for example, `toy_lm.vocab`.

In [15]:
## DO NOT MODIFY THIS CELL
from collections import namedtuple

def build_ngram_model(codefile, n):
    LanguageModel = namedtuple('LanguageModel', ['n', 'fd', 'vocab'])
    psents = process_code(codefile)
    vocab = get_vocab(psents)
    psentsunk = process_unk(psents, vocab)
    fd = get_freq_dict(psentsunk, n)
    return LanguageModel(n, fd, vocab)

toy_lm = build_ngram_model("../datasets/hw2/apache_spark.short.txt", 2)

### (b6) Log conditional probabilities. (10 points)

The heart of the language model above is just the frequency dictionary. To make our model functional, we will need to use the frequency dictionary to calculate (log) conditional probabilities. Write a function `log_prob(lm, context, token)` that takes in a language model `lm`, `context` in the form of a tuple, and a `token`. The function returns the *add-1 (Laplacian) smoothed* conditional probability values `log P(token|context)`.

We would like to calculate log probabilities, instead of raw probability values, because ultipmately we will use the language model to calculate the likelihood of a text. Multiplying many very small raw probability values may result in underflow issues (and inaccuracies) in any programming language.

You will need to get the size of the vocabulary when writing this function. **Keep in mind** that the `<UNK>`, `<s>`, and `</s>` symbols should all be a part of your vocabulary.

```
>>> log_prob(toy_lm, ('}',), '</s>')
-1.965301477025893
```

In [16]:
import math

def log_prob(lm, context, word) -> float:
    sample_size = lm.fd[context][word] + 1
    universe_size = sum(lm.fd[context].values()) + len(lm.vocab) + 3
    return math.log2(sample_size / universe_size)


In [17]:
# Do not change
log_prob(toy_lm, ('}',), '</s>')

-1.9653014770258928

### (b7) Perplexity (16 points)

Our final step to make our langauge model functional would be to calculate perplexity of a document (e.g., a new text file). Implement the function `get_ppl(lm, newfile)` that returns the perplexity of `newfile` given language model `lm` that you built using `build_ngram_model`.

Be mindful to check whether a word appears in the language model's vocabulary; if not, replace with `<UNK>`.

```
>>> get_ppl(toy_lm, "data/test.java.txt")
77.40867815861749
```

In [18]:
import math

def get_ppl(lm, testfile) -> float:
    lines = process_unk(process_code(testfile), lm.vocab)
    context_size = lm.n - 1
    P = N = 0
    for line in lines:
        N += 1 # I think the TA is counting <s> at the beggining as a token
        for i, token in enumerate(line + ['</s>']):
            padding = ['<s>'] * max(0, context_size - i)
            context = padding + line[max(0, i - context_size):i]
            P -= log_prob(lm, tuple(context), token)
            N += 1
    return 2**(P/N)


In [19]:
# Do not change
get_ppl(toy_lm, "../datasets/hw2/test.java.txt")

77.40867815856888

## (c) Project attribution (10 points)

If we have built language models of diffeernt projects, they can be used to check which project the unknown piece of code come from. Concretely, for the unknown lines of code, we can run the file through each known project's language model, and use perplexity to predict which project the unknown code is most likely to belong to.

In this problem, build two **bigram** models:
- a spark language model
- a gson language model

Make sure to train these models from the full text. Once you have trained both models, use the `get_ppl` function from each language model on the file `test.java.txt` to get the perplexity. 

Which project does the code come from?

In [23]:
spark_model = build_ngram_model("../datasets/hw2/apache_spark.large.txt", 2)
gson_model = build_ngram_model("../datasets/hw2/google_gson.large.txt", 2)

# Your code goes here: 

spark_ppl = get_ppl(spark_model, "../datasets/hw2/test.java.txt")
gson_ppl = get_ppl(gson_model, "../datasets/hw2/test.java.txt")
more_likely_project = "spark" if spark_ppl < gson_ppl else "gson"

In [24]:
# Do not change this

print("Test perplexity for spark: " + str(spark_ppl))
print("Test perplexity for gson: " + str(gson_ppl))
print("The more likely Project is " + more_likely_project)

Test perplexity for spark: 49.39873845439152
Test perplexity for gson: 125.53583007269333
The more likely Project is spark


## (d) Random code generator (5 points)

Now, we are ready to generate some Java functions! Starting with the start symbol `<s>`, at each step, use the previously generated token as context, and sample one of the top 5 tokens that follow this token. We stop whenever we hit `</s>`, or when we have generated a 100-token function, whichever is earlier.

Implement a function `code_generator(bigramlm, k)` that takes a bigram langauge model---trained on our apache_spark.large.txt---as input, and generates `k=5` random sentences.

**NOTE**: We do not want to generate `<UNK>`, so always remove `<UNK>` when it exists in the next tokens that follow the given context.
If you use the random seed 7 and randint to pick one token from the top 5 tokens, you will get the following result.

```
>>> funcs = code_generator(spark_model, 2)
>>> funcs[0]
'private static long ( ) , " + i = ( " ) ; return last comparison ; } if not found " ) { case 1 ; struct begin ( " ) , new illegal index ) , " a string get class ( new hash map ( ) , " a " a , 0 ] = = 0 ] = = = null ; if ( new illegal uao _ args struct field begin ( ) , 0 , string > 0 . get ( new hash set status ) , 0 ] = new tuple2 : '
```

In [38]:
from random import randint

def _get_most_common_tokens(lm, ctx, mc):
    return [token for token, count in lm.fd[tuple(ctx)].most_common(mc)]

def _generate_random_function(lm, tl=102, mc=5):
    context = ['<s>'] * (lm.n - 1)
    tokens = ['<s>']
    while (len(tokens) < tl) and (tokens[-1] != '</s>'):
        next_token = random.choice(_get_most_common_tokens(lm, context, mc))
        tokens.append(next_token)
        context = context[1:] + [next_token]
    return tokens


def code_generator(bigramlm, n) -> List[str]:
    return [_generate_random_function(bigramlm)[1:-1] for _ in range(n)]

In [39]:
# Do not change this 
import random
random.seed(7)

funcs = code_generator(spark_model, 5)
' '.join(funcs[0])

'private static long ( ) , " + i = ( " ) ; return last comparison ; } if not found " ) { case 1 ; struct begin ( " ) , new illegal index ) , " a string get class ( new hash map ( ) , " a " a , 0 ] = = 0 ] = = = null ; if ( new illegal uao _ args struct field begin ( ) , 0 , string > 0 . get ( new hash set status ) , 0 ] = new tuple2 :'